<center><h1>Classification by regional modeling</h1></center>

Classification by regional modeling consists in a five-step approach:
1. Setting the hyper-parameters. In this step, we specify the number of SOM prototypes $C$. It must be also defined as the maximum number of regions $K_{max}$. Without any prior knowledge, we will set in this example $K_{max} = \sqrt{C}$.


2. SOM training. In order to build regional models, follow the procedure introduced by Vesanto and Alhoniemi [1]. Thus, the very first step requires training the SOM as usual, with $C$ prototypes.


3. Clustering of the SOM. The step consists in performing clustering over the $C$ SOM prototypes. Although one may use any clustering algorithm for this step, for the sake of simplicity, we use the standard K-means algorithm in combination with the Davies–Bouldin (DB) index. The DB index is a clustering validity measure commonly used for finding the optimal number of clusters, but any suitable measure can be equally used (see [2]). Thus, we compute $K = 1, 2, ... K_{max}$ partitioning of the SOM prototypes and the corresponding DB index value as well. The optimal partitioning, represented by $K_{opt}$ partitions, is then the value of $K$ wich minimizes the DB index.


4. Partitioning SOM prototypes into regions. Once $K_{opt}$ is selected, the $r$-th cluster of SOM prototypes, $r = 1...K_{opt}$, is composed of all weight vectors $w_i$ that are mapped onto the prototype $p_r$ of the K-means algorithm. More formally, the set of SOM prototypes associated with the r-th prototype of the K-means algorithm is defined as:
$$W_r = \{w_i \in R^{p+q} | \|w_i-p_r\| < \|w_i-p_j\|, \forall j =1,...,K_{opt}, j\neq r \}$$


5. Mapping data points to regions. The fourth step consists in finding $K_{opt}$ data partitions, denoted by $\{X_1\}$, $\{X_2\}$, ... , $\{X_{K_{opt}}\}$ of the training dataset by mapping each datapoint to a region $r \in \{1, ... , K_{opt}\}$. In other words, let us denote $N_r$ as the number of data vectors in $\{X_r\}$. Then, the partition $\{X_r\}$ is composed of those input vectors $x_{rμ}$, $μ = 1, ... , N_r$ , whose closest SOM prototype belongs to $W_r$.


6. Building classification models over the regions. Finally, once the original dataset has been divided into $K_{opt}$ subsets (one per region), the last step consists in building $K_{opt}$ regional classification models using $X_r$, $r = 1, ... , K_{opt}$.

To test this framework the datasets below were gathered from the UCI repository:
* Vertebral Column
* Wall-Following
* Alzheimer (aquele usado na disciplina)

In [5]:
# loading datasets
import pandas as pd

# Vertebral Column
# dataset for classification between Normal (NO) and Abnormal (AB)
vc2c = pd.read_csv('datasets/vertebral_column/column_2C.dat', delim_whitespace=True, header=None)
# dataset for classification between DH (Disk Hernia), Spondylolisthesis (SL) and Normal (NO)
vc3c = pd.read_csv('datasets/vertebral_column/column_3C.dat', delim_whitespace=True, header=None)

# Wall-Following
# dataset with all 24 ultrassound sensors readings
wf24f = pd.read_csv('datasets/wall_following/sensor_readings_24.data', header=None)
# dataset with simplified 4 readings (front, left, right and back)
wf4f  = pd.read_csv('datasets/wall_following/sensor_readings_4.data',  header=None)
# dataset with simplified 2 readings (front and left)
wf2f  = pd.read_csv('datasets/wall_following/sensor_readings_2.data',  header=None)

# Parkinson (31 people, 23 with Parkinson's disease (PD))
temp = pd.read_csv('datasets/parkinson/parkinsons.data')
labels = temp.columns.values.tolist()
new_labels = [label for label in labels if label not in ('name')] # taking off column 'name'
pk = temp[new_labels]

In [6]:
pk_features = pk.columns.tolist()
pk_features.remove('status')

# datasets with separation between 'features' and 'labels'
datasets = {
    "vc2c":  {"features": vc2c.iloc[:,0:6],  "labels": pd.get_dummies(vc2c.iloc[:,6],  drop_first=True)},
    "vc3c":  {"features": vc3c.iloc[:,0:6],  "labels": pd.get_dummies(vc3c.iloc[:,6],  drop_first=True)},
    "wf24f": {"features": wf24f.iloc[:,0:24],"labels": pd.get_dummies(wf24f.iloc[:,24],drop_first=True)},
    "wf4f":  {"features": wf4f.iloc[:,0:4],  "labels": pd.get_dummies(wf4f.iloc[:,4],  drop_first=True)},
    "wf2f":  {"features": wf2f.iloc[:,0:2],  "labels": pd.get_dummies(wf2f.iloc[:,2],  drop_first=True)},
    "pk":    {"features": pk.loc[:,pk_features], "labels": pk.loc[:,["status"]]}
}

OBS: Was chosen to maintain k-1 dummies variables when we had k categories, so the missing category is identified when all dummies variables are zero.

## Step 1: Setting the hyper-parameters.
## Step 2: SOM training.

The code below implements the class *SOM* (self-organizing maps in a two-dimensional grid) and a function to plot data and neurons over all training iterations in the special case when the features space is also two-dimensional.

In [7]:
# New SOM
import numpy as np
import plotly.offline as plt
import plotly.graph_objs as go
from random import randint
import ipywidgets as widgets
from IPython.display import clear_output
from plotly import tools
from math import ceil

plt.init_notebook_mode(connected=True) # enabling plotly inside jupyter notebook

class SOM:
    'Class of Self Organizing Maps conected in a two-dimensional grid.'
    
    def __init__(self, nRows, nColumns): 
        self.nRows    = nRows
        self.nColumns = nColumns
        self.__epochs = 0 # number of epochs of trained SOM
        
        self.neurons = None
        self.indexes = None # list of index2D <=> index1D
        
        self.neuronsHist = None
        self.ssdHist     = None
        
    def init(self, X): # giving the data, so we can define maximum and minimum in each dimension
        # reset neuronsHist and ssdHist
        self.neuronsHist = None 
        self.ssdHist     = None
        
        dim = X.shape[1] # number of features
        rand = np.random.rand(self.nRows*self.nColumns, dim) # Auxiliary random element
        # find min-max for each dimension
        minimum = np.amin(X, axis=0)
        maximum = np.amax(X, axis=0)
        # Initializing neurons in random positions between minimum and maximum of the dataset
        self.neurons = (maximum - minimum)*rand + minimum
        
        # list of index2D == index1D
        self.indexes = self.index1Dto2D(np.arange(len(self.neurons))) 
        
    
    def fit(self, X, alpha0, sigma0, nEpochs=100, saveNeuronsHist=False, saveSSDHist=True, tol=1e-6,
              verboses=0, batchSize=np.inf):
        if self.neurons is None: self.init(X) # if self.init() wasn't run
        
        tau1 = nEpochs/sigma0
        tau2 = nEpochs
        SSD_new = self.SSD(X) # initial SSD, from random parameters
        
        if saveNeuronsHist: 
            self.neuronsHist    = [np.zeros(self.neurons.shape)]*(nEpochs+1)
            self.neuronsHist[0] = np.copy(self.neurons) # random neurons
        if saveSSDHist:
            self.ssdHist    = np.zeros((nEpochs+1))
            self.ssdHist[0] = SSD_new # initial SSD, from random parameters
        
        sigma = sigma0
        alpha = alpha0
        inertia = np.inf # initial value of inertia
        batchSize = min(len(X), batchSize) # adjusting ill defined batchSize
        for epoch in range(nEpochs):
            # Updating alpha and sigma
            sigma = sigma0*np.exp(-epoch/tau1);
            alpha = alpha0*np.exp(-epoch/tau2);
            
            order = np.random.permutation(len(X)) # shuffling order
            for i in order[:batchSize]: # for each datapoint in the shuflled order until batchSize
                # search for winner neuron
                winner_idx = self.get_winner(X[i])
                # neighbor function
                h_ik = self.h_neighbor(winner_idx, self.indexes, sigma)
                # updating neurons
                self.neurons += (h_ik[:, np.newaxis]*(X[i] - self.neurons))*alpha
                
            
            self.__epochs+=1 # updating number of epochs trained
            if verboses==1: print("End of epoch {}".format(epoch+1))
            
            SSD_old = SSD_new
            SSD_new = self.SSD(X)
            inertia = abs((SSD_old - SSD_new)/SSD_old)
            
            # Saving if necessary
            if saveNeuronsHist: self.neuronsHist[epoch+1] = np.copy(self.neurons)
            if saveSSDHist:     self.ssdHist[epoch+1]     = SSD_new
                       
            # breaking if tolerance reached before nEpochs
            if inertia < tol:
                # history cutting
                if saveNeuronsHist: self.neuronsHist = self.neuronsHist[0:epoch+2]
                if saveSSDHist:     self.ssdHist     = self.ssdHist[0:epoch+2]
                break
            
            
    def SSD(self, X):
        SSD = 0
        for x in X: # can it be vectorized?
            dist_2 = np.sum((self.neurons - x)**2, axis=1) # norm**2
            SSD += np.amin(dist_2)
        return SSD
        
    def index1Dto2D(self, index): # convert index of neurons parameter matrix to the 2D grid index
        return np.asarray([np.ceil((index+1)/self.nRows)-1, index%self.nRows], dtype='int').T
    
    def get_winner(self, x, dim=2, dist_matrix=False):
        dist_2 = np.sum((self.neurons - x)**2, axis=1) # norm**2
        
        temp = np.argmin(dist_2)
        winner = self.index1Dto2D(temp) if dim==2 else temp
        
        result = winner if dist_matrix==False else (winner, dist_2)
        return result
        
    def h_neighbor(self, idx_1, idx_2, sigma):
        dist_2 = np.sum((idx_2 - idx_1)**2, axis=1) # norm**2
        return np.exp( -dist_2/(2*sigma**2) )
    
    def getLabels(self, X, dim=1): # get labels in 1 dimension or 2 dimension (neuron grid)
        N = len(X)
        labels = np.zeros((N,dim), dtype='int')
        for i in range(N):
            labels[i,:] = self.get_winner(X[i,:],dim)
        return labels
    
    def plotSSD(self):
        traceData = go.Scatter(
            x = [i+1 for i in range(self.__epochs)], # epochs
            y = self.ssdHist, 
            mode='lines',
            name='SSD')
        data = [traceData]
        layoutData = go.Layout(
            title = "SSD history",
            xaxis=dict(title='Epoch'),
            yaxis=dict(title='SSD')
        )

        fig = go.Figure(data=data, layout=layoutData)
        plt.iplot(fig)
    
        
    def plotSOM(self, X=None):
        if self.neuronsHist is not None:
            # Int box to change the iteration number
            n_txt = widgets.BoundedIntText(
                value=0,
                min=0,
                max=len(self.neuronsHist)-1,
                step=10,
                description='epoch:'
            )    
        ###############################################################
        # Function to draw the graph
        def upgradeChart(change):
            clear_output()
            if self.neuronsHist is not None:
                display(n_txt)    
                n_ = change['new'] # new iteration number

            if self.neuronsHist is not None:
                x = self.neuronsHist[n_][:,0].tolist() 
                y = self.neuronsHist[n_][:,1].tolist()
                name = 'neurons [epoch ='+str(n_)+']'
            else:
                x = self.neurons[:,0].tolist()
                y = self.neurons[:,1].tolist()
                name = 'neurons'

            neurons = go.Scatter(x=x, y=y, mode='markers', name=name, 
                                 marker = dict(size=10,color = '#673AB7'))

            # cada linha que conecta os neurônios
            linhas = [{}]*(2*self.nRows*self.nColumns - self.nRows - self.nColumns)
            count=0 #contador para saber qual linha estamos
            for linha in range(self.nRows): # conecta da esquerda para direita
                for coluna in range(self.nColumns): # e de cima para baixo
                    try:
                        if self.neuronsHist is not None:
                            x0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna+1)).all(axis=1))[0][0], 0]
                            y1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna+1)).all(axis=1))[0][0], 1]
                        else:
                            x0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neurons[np.where((self.indexes==(linha,coluna+1)).all(axis=1))[0][0], 0]
                            y1 = self.neurons[np.where((self.indexes==(linha,coluna+1)).all(axis=1))[0][0], 1]

                        linhas[count]= {'type':'line','x0':x0,'y0': y0,'x1':x1,'y1':y1,
                                        'line': {'color': '#673AB7','width': 1,}}
                        count+=1
                    except: # edge of the grid
                        pass
                    try:
                        if self.neuronsHist is not None:
                            x0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha, coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha+1, coluna)).all(axis=1))[0][0], 0]
                            y1 = self.neuronsHist[n_][np.where(
                                (self.indexes==(linha+1, coluna)).all(axis=1))[0][0], 1]
                        else:
                            x0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   0]
                            y0 = self.neurons[np.where((self.indexes==(linha,coluna)).all(axis=1))[0][0],   1]
                            x1 = self.neurons[np.where((self.indexes==(linha+1,coluna)).all(axis=1))[0][0], 0]
                            y1 = self.neurons[np.where((self.indexes==(linha+1,coluna)).all(axis=1))[0][0], 1]

                        linhas[count] = {'type': 'line','x0': x0,'y0': y0,'x1': x1,'y1': y1,
                                         'line': {'color': '#673AB7','width': 1}}
                        count+=1
                    except: # edge of the grid
                        pass

            data = []
            title = ""
            if X is not None:
                datapoints = go.Scatter(x = X[:,0], y = X[:,1], mode='markers', name='data',
                                        marker = dict(size = 5,color = '#03A9F4'))
                data = [datapoints, neurons]
                title = "Data + SOM"
            else:
                data = [neurons]
                title = "SOM"

            layout = go.Layout(title=title, xaxis=dict(title="$x_1$"), yaxis=dict(title="$x_2$"),
                               shapes=linhas)

            fig = go.Figure(data=data, layout=layout)
            plt.iplot(fig)
        ###########################################################################

        if self.neuronsHist is not None: n_txt.observe(upgradeChart, names='value')
        upgradeChart({'new': 0})

Function to scale data:

In [8]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Função para mudar a escala dos dados
def scale_feat(X_train, X_test, scaleType='min-max'):
    if scaleType=='min-max' or scaleType=='std':
        X_tr_norm = np.copy(X_train) # fazendo cópia para deixar original disponível
        X_ts_norm = np.copy(X_test)
        scaler = MinMaxScaler() if scaleType=='min-max' else StandardScaler()
        scaler.fit(X_tr_norm)
        X_tr_norm = scaler.transform(X_tr_norm)
        X_ts_norm = scaler.transform(X_ts_norm)
        return (X_tr_norm, X_ts_norm)
    else:
        raise ValueError("Tipo de escala não definida. Use 'min-max' ou 'std'.")
        
import datetime
def printDateTime():
    print(datetime.datetime.now())

Example of class `SOM` runing:

In [26]:
%%time
import datetime
print(datetime.datetime.now())

nEpochs = 30
print("nEpochs = {}".format(nEpochs))

data = datasets['wf2f']
N = len(data['features'].index) # number of datapoints
l = ceil((5*N**.5)**.5) # tamanho do lado da grid quadrada de neurônios
X = data['features'].values.copy()

X1, X2 = scale_feat(X,X,scaleType='min-max')
X=X1

som = SOM(l, l)
som.fit(X=X, alpha0=0.1, sigma0=3, nEpochs=nEpochs, saveNeuronsHist=True, verboses=1)
print(som.ssdHist[-1])

2019-07-05 22:28:46.360346
nEpochs = 30
End of epoch 1
End of epoch 2
End of epoch 3
End of epoch 4
End of epoch 5
End of epoch 6
End of epoch 7
End of epoch 8
End of epoch 9
End of epoch 10
End of epoch 11
End of epoch 12
End of epoch 13
End of epoch 14
End of epoch 15
End of epoch 16
End of epoch 17
End of epoch 18
End of epoch 19
End of epoch 20
End of epoch 21
End of epoch 22
End of epoch 23
End of epoch 24
End of epoch 25
End of epoch 26
End of epoch 27
End of epoch 28
End of epoch 29
End of epoch 30
0.12000278956169355
CPU times: user 13.9 s, sys: 32.6 ms, total: 13.9 s
Wall time: 13.8 s


In [27]:
som.plotSSD()

In [28]:
som.plotSOM(X)

BoundedIntText(value=0, description='epoch:', max=30, step=10)

The code below trains the SOM's in all datasets:

Note: The number of neurons chosen was approximately $5\sqrt{N}$ of the dataset and a square grid to arrange them.

In [29]:
%%time
printDateTime()

nEpochs = 100
soms = {}
for name, data in datasets.items():
    N = len(data['features'].index) # number of datapoints
    l = ceil((5*N**.5)**.5) # side length of square grid of neurons
    X = data['features'].values
    
    # scaling dataset
    X1, X2 = scale_feat(X,X,scaleType='min-max')
    X=X1
    
    # SOM training
    som = SOM(l,l)
    som.fit(X=X, alpha0=0.1, sigma0=3, nEpochs=nEpochs)
    
    soms[name] = som
    
    print("{} done!".format(name))
    
# CPU times: user 3min 10s, sys: 114 ms, total: 3min 10s
# Wall time: 3min 10s

2019-07-05 22:29:01.564994
vc2c done!
vc3c done!
wf24f done!
wf4f done!
wf2f done!
pk done!
CPU times: user 3min 4s, sys: 522 ms, total: 3min 4s
Wall time: 3min 3s


In [30]:
for dataset in datasets:
    print(dataset)
    soms[dataset].plotSSD()

vc2c


vc3c


wf24f


wf4f


wf2f


pk


# Step 3: Clustering of the SOM.

Function (and help functions) that implement Davies–Bouldin (DB) validation index:

In [13]:
from numpy.linalg import norm

def DB(model, X, q=2, t=2):
    k = len(model.cluster_centers_) # number of clusters
    db = 0
    for i in range(k):
        db += R(i,q,t,model,X)
    db/=k
    return db

def R(i,q,t,model,X):
    js = [j for j in range(len(model.cluster_centers_)) if j!=i] # for j!=i
    R_iqt = 0 # R_iqt is always >= 0
    for j in js:
        temp = (S(i,q,model,X) + S(j,q,model,X)) / d(i,j,t,model)
        R_iqt = temp if temp > R_iqt else R_iqt # searching for the maximum
    return R_iqt

def S(i,q,model,X):
    Vi = X[np.where(model.labels_ == i)] # partition 'i'
    wi = model.cluster_centers_[i]       # cluster center of partition 'i'
    S_iq = 0
    for x in Vi:
        S_iq += norm(x-wi)**q
    
    S_iq = (S_iq/len(Vi)) ** (1/q)
    return S_iq

def d(i,j,t,model):
    wi = model.cluster_centers_[i]
    wj = model.cluster_centers_[j]
    d_ijt = 0
    for m in range(len(wi)):
        d_ijt += (abs(wi[m] - wj[m])) ** t
    d_ijt = d_ijt ** (1/t)
    return d_ijt

In [14]:
from numpy import trace

def CH(kmeans, points):
    # ks is the vector of clusters. Ex: [0 1 2 3 4]
    # N is the vector frequency of each label. Ex: [14 10 5 12]
    ks, N = np.unique(np.sort(kmeans.labels_), return_counts=True)
    x_bar = np.mean(points, axis=0).reshape(-1,1)
    
    # matriz dispersão entregrupos
    Bk = np.zeros((points.shape[1],points.shape[1]))
    # matriz dispersão intragrupo
    Wk = np.zeros((points.shape[1],points.shape[1]))
    for i in ks: # para cada protótipo
        wi = kmeans.cluster_centers_[i].reshape(-1,1) # protótipo do cluster 'i'
        temp = wi - x_bar
        Bk += N[i]*np.matmul(temp,temp.T)
        
        Vi = points[kmeans.labels_==i].T
        for l in range(Vi.shape[1]): # para cada elemento da partição 'i'
            temp = Vi[:,l] - wi
            Wk += np.matmul(temp,temp.T)
            
    N = len(points) # number of points
    K = len(kmeans.cluster_centers_) # number os clusters
    ch = ( trace(Bk)/(K-1) ) / ( trace(Wk)/(N-K) )
    return ch

In [15]:
import base

Clustering and searching for optimal $k$ in range 2 to $\sqrt{C}$, where $C$ is the number os SOM prototypes:

In [8]:
%%time
    
from sklearn.cluster import KMeans

printDateTime()
validation_indices = {
    'DB':   {},
    'Dunn': {},
    'CH':   {}
}

for dataset_name in datasets:
        som = soms[dataset_name]
        C = len(som.neurons)
        ks = [i for i in range(2, ceil(C**(1/2)))] # range to search for k in k-means

        n_init = 10 # number of independent rounds of initialization
        validation_indices['DB'][dataset_name]   = [0]*len(ks)
        validation_indices['Dunn'][dataset_name] = [0]*len(ks)
        validation_indices['CH'][dataset_name]   = [0]*len(ks)
        for i in range(len(ks)):
            kmeans = KMeans(n_clusters=ks[i], n_init=n_init, init='random', n_jobs=-1).fit(som.neurons)
            # test if number of distinct clusters == number of clusters specified
            centroids = kmeans.cluster_centers_
            if len(centroids) == len(np.unique(centroids,axis=0)):
                validation_indices['DB'][dataset_name][i] = DB(kmeans,som.neurons)
            else:
                validation_indices['DB'][dataset_name][i] = np.inf

            validation_indices['Dunn'][dataset_name][i] = base.dunn_fast(som.neurons, kmeans.labels_)
            validation_indices['CH'][dataset_name][i]   = CH(kmeans, som.neurons)

        print("End of dataset {}".format(dataset_name))

2019-07-06 14:53:05.423260


NameError: name 'soms' is not defined

In [35]:
def plot_validation_indices(dataset_name, validation_indices):
    data = []
    for index_name, results_vec in validation_indices.items():
    #for validation_index in validation_indices:
        #print(index_name)
        #print(results_vec[dataset_name])
        data.append(go.Scatter(
            x=[i for i in range(2, len(results_vec[dataset_name])+2)],
            y=results_vec[dataset_name], 
            mode='lines+markers', 
            name="{} index".format(index_name)))

    
    layout = go.Layout(
        title = "Indices vs k [{} dataset]".format(dataset_name),
        legend=dict(orientation="h", y=-.05),
        xaxis=dict(title="Number of clusters (k)"),
        yaxis=dict(title="Indices values")
    )

    fig = go.Figure(data=data, layout=layout)
    plt.iplot(fig)


for dataset_name in datasets:
    plot_validation_indices(dataset_name, validation_indices)
    #plot_db(db[dataset_name], dataset_name)
    for index_name, results_vec in validation_indices.items():
        results = results_vec[dataset_name]
        k_opt = np.argmin(results) if index_name=='DB' else np.argmax(results)
        k_opt += 2
        print("K_opt for {} dataset using {} index: {}".format(
              dataset_name, index_name, k_opt))
        
    #print("K_opt for {} dataset is: {}".format(dataset_name, np.argmin(db[dataset_name])+2))

K_opt for vc2c dataset using DB index: 4
K_opt for vc2c dataset using Dunn index: 8
K_opt for vc2c dataset using CH index: 2


K_opt for vc3c dataset using DB index: 2
K_opt for vc3c dataset using Dunn index: 7
K_opt for vc3c dataset using CH index: 2


K_opt for wf24f dataset using DB index: 17
K_opt for wf24f dataset using Dunn index: 7
K_opt for wf24f dataset using CH index: 2


K_opt for wf4f dataset using DB index: 5
K_opt for wf4f dataset using Dunn index: 19
K_opt for wf4f dataset using CH index: 2


K_opt for wf2f dataset using DB index: 5
K_opt for wf2f dataset using Dunn index: 3
K_opt for wf2f dataset using CH index: 2


K_opt for pk dataset using DB index: 2
K_opt for pk dataset using Dunn index: 7
K_opt for pk dataset using CH index: 2


In [8]:
def plot_kmeans(kmeans, X):
    data = []
    if X is not None:
        datapoints = go.Scatter(
            x = X[:,0], 
            y = X[:,1], 
            mode='markers',
            name='data',
            marker = dict(
                 size = 5,
                 color = '#03A9F4'
                )
        )
        data.append(datapoints)
    
    kmeans_clusters = go.Scatter(
        x=kmeans.cluster_centers_[:,0],
        y=kmeans.cluster_centers_[:,1], 
        mode='markers', 
        name='kmeans clusters', 
        marker = dict(size=10,color = '#673AB7')
    )
    data.append(kmeans_clusters)

    layout = go.Layout(
        title = "Data + KMeans clusters",
        xaxis=dict(title="$x_1$"),
        yaxis=dict(title="$x_2$"),
    )

    fig = go.Figure(data=data, layout=layout)
    plt.iplot(fig)

plot_kmeans(kmeans,som.neurons)

NameError: name 'kmeans' is not defined

# Step 4: Partitioning SOM prototypes into regions:

# Step 4: Mapping data points to regions:

# Step 6: Building classification models over the regions.

Below is the class Regional Linear Model wich implements the OLS in the regional fashion.

In [9]:
class BiasModel:
    'Class the implements a dummy model in case of homogenous region'
    
    def __init__(self, class_label):
        self.class_label = class_label
        
    def predict(self, X):
        return np.vstack( [self.class_label]*len(X) )

In [22]:
from sklearn import linear_model
from sklearn.cluster import KMeans
from copy import copy

class RegionalModel:
    'Class of Regional Models.'
    
    def __init__(self, SOM_class, Model_class, Cluster_class=None):
        self.SOM             = SOM_class
        self.Cluster         = Cluster_class
        self.Model           = Model_class
        self.region_labels   = []
        self.regional_models = []
        self.empty_regions   = []
        self.targets_dim_    = None
        

    def fit(self, X, Y, verboses=0, SOM_params=None, Cluster_params=None, Model_params=None):
        self.targets_dim_ = Y.shape[1] # dimension of target values
        
        # SOM training
        if SOM_params is not None:
            if verboses==1: print("Start of SOM training at {}".format(datetime.datetime.now()))
            self.SOM.fit(X=X, **SOM_params)
        
        # Cluster training
        if Cluster_params is not None:
            if verboses==1: print("Start of clustering SOM prototypes at {}".format(datetime.datetime.now()))
            
            # Search for k_opt
            k_opt = None
            if type(Cluster_params['n_clusters']) is dict: # a search is implied:
                eval_function = Cluster_params['n_clusters']['metric']
                find_best     = Cluster_params['n_clusters']['criteria']
                k_values      = Cluster_params['n_clusters']['k_values']
                
                validation_index = [0]*len(k_values)
                for i in range(len(k_values)):
                    kmeans = KMeans(n_clusters=k_values[i],
                                    n_init=10,
                                    init='random'
                                    #n_jobs=-1
                                   ).fit(self.SOM.neurons)
                    # test if number of distinct clusters == number of clusters specified
                    centroids = kmeans.cluster_centers_
                    if len(centroids) == len(np.unique(centroids,axis=0)):
                        validation_index[i] = eval_function(kmeans,self.SOM.neurons)
                    else:
                        validation_index[i] = np.NaN
                
                k_opt = k_values[find_best(validation_index)]
                if verboses==1: print("Best k found: {}".format(k_opt))
            else:
                k_opt = Cluster_params['n_clusters']
            
            params = Cluster_params.copy()
            del params['n_clusters'] # deleting unecessary param
            # real training of clustering algorithm
            self.Cluster = KMeans(n_clusters=k_opt, **params).fit(self.SOM.neurons)
            
        
        # Model training
        self.region_labels = self.regionalize(X) # finding labels of datapoints
        if verboses==1: print("Start of Model training at {}".format(datetime.datetime.now()))
        
        self.regional_models = [None]*k_opt    
        for r in range(k_opt): # for each region
            Xr = X[np.where(self.region_labels == r)[0]]
            Yr = Y[np.where(self.region_labels == r)[0]]
            
            model_r = None
            if len(Xr) == 0: # empty region
                self.empty_regions.append(r)
            
            elif len( np.unique(Yr, axis=0) ) == 1: # homogenous region, only one class
                model_r = BiasModel(np.unique(Yr, axis=0))
                
            else: # region not empty or homogeneous
                self.Model.fit(Xr,Yr)
                model_r = copy(self.Model)

            self.regional_models[r] = model_r
            
                       
                
    def regionalize(self, X):
        regions = np.zeros(len(X), dtype='int')
        for i in range(len(X)): # for each datapoint
            winner_som_idx, dist_matrix = self.SOM.get_winner(
                                          X[i], dim=1, dist_matrix=True) # find closest neuron
            region = self.Cluster.labels_[winner_som_idx] # find neuron label index in kmeans
            
            # if the region don't have a model is because it didn't have datapoints in the train set
            if region in self.empty_regions: 
                dead_neurons, = np.where(self.Cluster.labels_==region)
                dist_matrix[dead_neurons] = np.inf # taking off dead neurons from the play
                
                temp = np.argmin(dist_matrix)
                region = self.Cluster.labels_[temp]
                
            regions[i] = region 
        
        return regions
          
    
    def predict(self, X):
        # searching for a non-empty region
        regions = [i for i in range(self.Cluster.n_clusters)]
        not_empty_regions = list(set(regions) - set(self.empty_regions))
        
#         temp = self.regional_models[not_empty_regions[0]].intercept_
        predictions = np.zeros((
            len(X),           # number of samples
            self.targets_dim_ # size of output Y
        ))
        
        regions = self.regionalize(X)
        for i in range(len(X)):
            predictions[i,:] = self.regional_models[regions[i]].predict(X[i].reshape(1, -1))
        
        return predictions


In [23]:
# convert dummies to multilabel
def dummie_to_multilabel(X):
    N = len(X)
    X_multi = np.zeros((N,1),dtype='int')
    for i in range(N):
        temp = np.where(X[i]==1)[0] # find where 1 is found in the array
        if temp.size == 0: # is a empty array, there is no '1' in the X[i] array
            X_multi[i] = 0 # so we denote this class '0'
        else:
            X_multi[i] = temp[0] + 1 # we have +1 because 
    return X_multi.T[0]

Example of class `RegionalModel` running with linear models:

In [28]:
%%time
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.cluster import KMeans


dataset_name='wf2f'

X = datasets[dataset_name]['features'].values
Y = datasets[dataset_name]['labels'].values

# Train/Test split = 80%/20%
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

# scaling features
X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max')

#N = len(dataset['features'].index) # number of datapoints
N = len(X_tr_norm) # number of datapoints in the train split
l = ceil((5*N**.5)**.5) # side length of square grid of neurons

som = SOM(l,l)
som_params={
    'alpha0':    0.01,
    'sigma0':    1,
    'nEpochs':   1,
    'verboses':  0            
}

C = l**2 # number of SOM neurons in the 2D grid
k_values = [i for i in range(2, ceil(np.sqrt(C)))] # 2 to sqrt(C)
cluster_params={
    'n_clusters': {'metric':   DB,        # when a dictionary is pass a search begins
                   'criteria': np.argmin, # search for smallest DB score 
                   'k_values': k_values}, # around the values provided in 'k_values'
    'n_init':     10, # number of initializations
    'init':       'random', 
    #'n_jobs':     -1
}

linearModel = linear_model.LinearRegression(n_jobs=-1)

rm = RegionalModel(som, linearModel)
rm.fit(X=X_tr_norm, Y=y_train, verboses=0,
        SOM_params     = som_params,
        Cluster_params = cluster_params)

# Evaluating in the test dataset
y_pred = rm.predict(X_ts_norm)
y_pred = np.round(np.clip(y_pred, 0, 1)) # rounding prediction numbers

cm = confusion_matrix(dummie_to_multilabel(y_test),
                      dummie_to_multilabel(y_pred))
#cm = np.asarray(cm).reshape(-1) # matrix => array
acc=0
total=sum(sum(cm))
for j in range(len(cm)):
    acc += cm[j,j] # summing the diagonal
acc/=total

CPU times: user 1.69 s, sys: 2.98 ms, total: 1.69 s
Wall time: 1.69 s


In [29]:
acc

0.8113553113553114

Evaluation of regional OLS in the datasets:

In [113]:
# constant hyperparameters:
test_size = 0.2
scaleType = 'min-max'
n_resamplings = 100

# hyperparameters grid search:
num = 3
alphas = np.linspace(0.1, 0.5,  num=num).tolist()
sigmas = np.linspace(3,    10,   num=num).tolist()
epochs = np.linspace(100,  500, num=num, dtype='int').tolist()

# vector of random states for train/test split
random_states = np.random.randint(np.iinfo(np.int32).max, size=n_resamplings).tolist()
cases = [
    {
         "dataset_name" : dataset_name
        ,"random_state":  random_state
        ,"som_params":    { "alpha0"  : alpha0
                           ,"sigma0"  : sigma0
                           ,"nEpochs" : nEpochs
                          }
    }
    # hyperparameters possible values
    for dataset_name in datasets.keys()
    for random_state in random_states
    for alpha0       in alphas
    for sigma0       in sigmas
    for nEpochs      in epochs
]

print("alphas: {}\nsigmas: {}\nepochs: {}\n".format(alphas,sigmas,epochs))

print("# of alphas: {}\n# of sigmas: {}\n# of epochs: {}\n# of random_states: {}\n# of datasets: {}\n".format(
    len(alphas), len(sigmas), len(epochs), len(random_states), len(list(datasets.keys()))))

print("# of cases: {}".format(len(cases)))

alphas: [0.1, 0.30000000000000004, 0.5]
sigmas: [3.0, 6.5, 10.0]
epochs: [100, 300, 500]

# of alphas: 3
# of sigmas: 3
# of epochs: 3
# of random_states: 100
# of datasets: 6

# of cases: 16200


In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from functools import partial
from multiprocessing import Pool

def evalRLM(case):
    dataset_name = case['dataset_name']
    random_state = case['random_state']
    som_params   = case['som_params']
    
    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values
    
    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=test_size, random_state=random_state)
    # scaling features
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scaleType)

    N = len(X_tr_norm) # number of datapoints in the train split
    l = ceil((5*N**.5)**.5) # side length of square grid of neurons

    som = SOM(l,l)

    C = l**2 # number of SOM neurons in the 2D grid
    k_values = [i for i in range(2, ceil(np.sqrt(C)))] # 2 to sqrt(C)
    cluster_params={
        'n_clusters': {'metric':   DB,        # when a dictionary is pass a search begins
                       'criteria': np.argmin, # search for smallest DB score 
                       'k_values': k_values}, # around the values provided in 'k_values'
        'n_init':     10, # number of initializations
        'init':       'random'
        #'n_jobs':     0
    }

    linearModel = linear_model.LinearRegression()

    rlm = RegionalModel(som, linearModel)
    rlm.fit(X=X_tr_norm, Y=y_train,
            SOM_params     = som_params,
            Cluster_params = cluster_params)

    # Evaluating in the train set
    y_tr_pred = rlm.predict(X_tr_norm)
    y_tr_pred = np.round(np.clip(y_tr_pred, 0, 1)) # rounding prediction numbers

    cm_tr = confusion_matrix(dummie_to_multilabel(y_train),
                             dummie_to_multilabel(y_tr_pred)
                            ).reshape(-1) # matrix => array

    # Evaluating in the test set
    y_ts_pred = rlm.predict(X_ts_norm)
    y_ts_pred = np.round(np.clip(y_ts_pred, 0, 1)) # rounding prediction numbers

    cm_ts = confusion_matrix(dummie_to_multilabel(y_test),
                             dummie_to_multilabel(y_ts_pred)
                            ).reshape(-1) # matrix => array

    data = [dataset_name, random_state]+list(som_params.values())+[cm_tr]+[cm_ts]
    return data

In [8]:
# browser notification when cell finishs with '%%notify'
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [15]:
%%notify
from multiprocessing import Pool
import tqdm

data = [None]*len(cases)
count=0
pool = Pool()
for i in tqdm.tqdm(pool.imap_unordered(evalRLM, cases), total=len(cases)):
    data[count] = i
    count+=1
pool.close()
pool.join()


results = np.vstack(data)
header  = ["dataset_name", "random_state", "alpha0", "sigma0", "nEpochs", "cm_tr", "cm_ts"]
results_df = pd.DataFrame(results, columns=header)

filename = "ROLS - all - n_res={n_resamplings} - {datetime}.csv".format(
    n_resamplings=n_resamplings,
    datetime=datetime.datetime.now()
)
results_df.to_csv(filename,index=False) # saving results in csv file

# [{elapsed}<{remaining}

100%|██████████| 16200/16200 [84:53:44<00:00,  1.07s/it]   


<IPython.core.display.Javascript object>

### Processing results:

In [111]:
# loading simulation results
df_results = pd.read_csv('ROLS - all - n_res=100 - 2019-07-10 04:50:57.404253.csv')
df_results.head()

,dataset_name,random_state,alpha0,sigma0,nEpochs,cm_tr,cm_ts
0,vc2c,127815836,0.1,3.0,100,[156 13 15 64],[32 9 7 14]
1,vc2c,127815836,0.1,10.0,100,[153 16 16 63],[32 9 6 15]
2,vc2c,127815836,0.1,6.5,100,[162 7 21 58],[34 7 9 12]
3,vc2c,127815836,0.1,10.0,300,[157 12 21 58],[32 9 8 13]
4,vc2c,127815836,0.1,6.5,300,[155 14 19 60],[33 8 7 14]


Results for each data set:

In [148]:
som_params = [
    {
     "alpha0"  : alpha0
    ,"sigma0"  : sigma0
    ,"nEpochs" : nEpochs
    }
    for alpha0       in alphas
    for sigma0       in sigmas
    for nEpochs      in epochs
]

header = list(som_params[0].keys()) + ['Minimum', 'Maximum', 'Median', 'Mean', 'Std. Deviation']

df_ds = {}
for dataset_name in datasets: # For this specific dataset
    print(dataset_name)
    df = df_results.loc[df_results['dataset_name'] == dataset_name] # get simulation results

    count = 0
    df_data   = np.zeros((len(som_params), len(header))) # matriz que guardará resultados numéricos
    for params in som_params:
        df_case = df.loc[(df['alpha0']  == params['alpha0']) & 
                         (df['sigma0']  == params['sigma0']) &
                         (df['nEpochs'] == params['nEpochs'])]

        # converting confusion matrix from string to numpy array
        cm_ts = np.array([[int(x) for x in result[1:-1].split()] for result in df_case['cm_ts'].values])

        #data = cm_ts
        length = cm_ts.shape[1]
        cm_side = int(np.sqrt(length))

        acc   = [0]*len(cm_ts)
        for i in range(len(cm_ts)):
            cm = np.reshape(cm_ts[i], (cm_side,cm_side))
            acc[i] = np.trace(cm)/np.sum(cm)

        df_data[count,:] = np.matrix([
            params['alpha0'], params['sigma0'], params['nEpochs'],
            min(acc), max(acc), np.median(acc), np.mean(acc), np.std(acc)])
        count+=1


    df_ds[dataset_name] = pd.DataFrame(df_data, columns=header)
    display(df_ds[dataset_name].head())
    print('-'*100,'\n'*2)

vc2c


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.677419,0.919355,0.838710,0.827097,0.050134
1,0.1,3.0,300.0,0.725806,0.935484,0.838710,0.829516,0.045291
2,0.1,3.0,500.0,0.709677,0.919355,0.838710,0.828710,0.043863
3,0.1,6.5,100.0,0.693548,0.903226,0.838710,0.828871,0.046573
4,0.1,6.5,300.0,0.709677,0.935484,0.830645,0.828871,0.045785


---------------------------------------------------------------------------------------------------- 


vc3c


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.645161,0.919355,0.822581,0.811129,0.049624
1,0.1,3.0,300.0,0.709677,0.919355,0.822581,0.814194,0.046326
2,0.1,3.0,500.0,0.645161,0.919355,0.806452,0.815484,0.051930
3,0.1,6.5,100.0,0.693548,0.919355,0.814516,0.810806,0.044748
4,0.1,6.5,300.0,0.709677,0.919355,0.822581,0.818226,0.044573


---------------------------------------------------------------------------------------------------- 


wf24f


...

---------------------------------------------------------------------------------------------------- 


wf2f


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.769231,0.960623,0.908883,0.911016,0.023783
1,0.1,3.0,300.0,0.822344,0.956044,0.905220,0.905302,0.021214
2,0.1,3.0,500.0,0.812271,0.955128,0.904762,0.907152,0.021152
3,0.1,6.5,100.0,0.855311,0.957875,0.907509,0.907326,0.017636
4,0.1,6.5,300.0,0.830586,0.954212,0.906136,0.907115,0.015866


---------------------------------------------------------------------------------------------------- 


pk


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.717949,1.000000,0.871795,0.860513,0.059177
1,0.1,3.0,300.0,0.692308,0.974359,0.871795,0.870513,0.056686
2,0.1,3.0,500.0,0.692308,0.974359,0.871795,0.859487,0.056466
3,0.1,6.5,100.0,0.692308,1.000000,0.871795,0.865128,0.062030
4,0.1,6.5,300.0,0.692308,1.000000,0.871795,0.856923,0.058939


---------------------------------------------------------------------------------------------------- 




Taking the best values by higher mean in accuracy.

In [149]:
data = np.array([df.sort_values('Mean', ascending=False).iloc[0,:].values for df in df_ds.values()])
idx_label = list(df_ds.keys())
df_rols = pd.DataFrame(data, columns=header, index=[idx_label])
df_rols

,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
vc2c,0.1,10.0,100.0,0.725806,0.951613,0.838710,0.832419,0.045040
vc3c,0.1,6.5,300.0,0.709677,0.919355,0.822581,0.818226,0.044573
wf24f,0.5,10.0,300.0,0.825092,0.886447,0.859890,0.860018,0.012187
wf4f,0.5,6.5,300.0,0.850733,0.935897,0.878205,0.883727,0.020317
wf2f,0.1,10.0,300.0,0.849817,0.960623,0.908425,0.911868,0.017521
pk,0.1,3.0,300.0,0.692308,0.974359,0.871795,0.870513,0.056686


# Globlal OLS

In [116]:
# constant hyperparameters:
test_size = 0.2
scaleType = 'min-max'
n_resamplings = 100

# vector of random states for train/test split
random_states = np.random.randint(np.iinfo(np.int32).max, size=n_resamplings).tolist()
cases = [
    {
         "dataset_name" : dataset_name
        ,"random_state":  random_state
    }
    for dataset_name in datasets.keys()
    for random_state in random_states
]

print("# of random_states: {}\n# of datasets: {}\n".format(
    len(random_states), len(list(datasets.keys()))))

print("# of cases: {}".format(len(cases)))

# of random_states: 100
# of datasets: 6

# of cases: 600


In [75]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import linear_model

def evalGOLS(case):
    dataset_name = case['dataset_name']
    random_state = case['random_state']
    
    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values
    
    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=test_size, random_state=random_state)

    # scaling features
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max')

    model = linear_model.LinearRegression().fit(X_tr_norm,y_train)
    
    # Evaluating in the train set
    y_tr_pred = model.predict(X_tr_norm)
    y_tr_pred = np.round(np.clip(y_tr_pred, 0, 1)) # rounding prediction numbers

    cm_tr = confusion_matrix(dummie_to_multilabel(y_train),
                             dummie_to_multilabel(y_tr_pred)
                            ).flatten() # matrix => array
    
    # Evaluating in the test set
    y_ts_pred = model.predict(X_ts_norm)
    y_ts_pred = np.round(np.clip(y_ts_pred, 0, 1)) # rounding prediction numbers

    cm_ts = confusion_matrix(dummie_to_multilabel(y_test),
                             dummie_to_multilabel(y_ts_pred)
                            ).flatten() # matrix => array

    
    data = [dataset_name, random_state]+[cm_tr]+[cm_ts]
    return data

In [76]:
from multiprocessing import Pool
import tqdm

data = [None]*len(cases)

pool = Pool()
data =[result for result in tqdm.tqdm(pool.imap_unordered(evalGOLS,cases), total=len(cases))]
pool.close()
pool.join()

results = np.vstack(data)
header  = ["dataset_name", "random_state", "cm_tr", "cm_ts"]
results_df = pd.DataFrame(results, columns=header)

100%|██████████| 600/600 [00:07<00:00, 82.56it/s] 


In [77]:
results_df.head()

,dataset_name,random_state,cm_tr,cm_ts
0,vc2c,696399911,"[162, 10, 32, 44]","[37, 1, 8, 16]"
1,vc2c,665400398,"[156, 12, 27, 53]","[36, 6, 5, 15]"
2,vc2c,1026613726,"[148, 17, 18, 65]","[39, 6, 4, 13]"
3,vc2c,1775733262,"[156, 15, 28, 49]","[37, 2, 7, 16]"
4,vc2c,406143792,"[163, 10, 28, 47]","[33, 4, 10, 15]"


Processing results (taking the best values by higher mean in accuracy):

In [150]:
header = ['Minimum', 'Maximum', 'Median', 'Mean', 'Std. Deviation']

data      = np.zeros(( len(datasets.keys()), len(header) ))
idx_label = [' ']*len(datasets.keys())
count=0
for dataset_name in datasets: # For this specific dataset
    df = results_df.loc[results_df['dataset_name'] == dataset_name] # get simulation results
    
    # converting confusion matrices to numpy matrix
    cm_ts = np.array([array for array in df['cm_ts'].values])
       
    length = cm_ts.shape[1]
    cm_side = int(np.sqrt(length))
    acc   = [0]*len(cm_ts)
    for i in range(len(cm_ts)):
        cm = np.reshape(cm_ts[i], (cm_side,cm_side))
        acc[i] = np.trace(cm)/np.sum(cm)

    data[count,:] = np.array([min(acc), max(acc), np.median(acc), np.mean(acc), np.std(acc)])
    idx_label[count] = dataset_name
    count+=1
    
df_ols = pd.DataFrame(data, columns=header, index=[idx_label])
df_ols

,Minimum,Maximum,Median,Mean,Std. Deviation
vc2c,0.725806,0.919355,0.838710,0.833387,0.041377
vc3c,0.661290,0.903226,0.774194,0.778226,0.047867
wf24f,0.600733,0.666667,0.638278,0.638022,0.013931
wf4f,0.700549,0.761905,0.727564,0.727793,0.012644
wf2f,0.691392,0.759158,0.722070,0.722976,0.013688
pk,0.692308,0.974359,0.871795,0.866667,0.049720


# Comparing results:

In [171]:
header = ['Dataset', 'Model']+list(df_ols.columns)

temp_rols = df_rols.rename_axis('Dataset').reset_index().loc[:,[x for x in header if x!='Model']]
temp_rols.insert(1,'Model',['ROLS']*len(datasets.keys()))

temp_ols = df_ols.rename_axis('Dataset').reset_index()
temp_ols.insert(1,'Model',['OLS']*len(datasets.keys()))

display(
    pd.concat([temp_ols,temp_rols]).sort_index()
)

,Dataset,Model,Minimum,Maximum,Median,Mean,Std. Deviation
0,vc2c,OLS,0.725806,0.919355,0.838710,0.833387,0.041377
0,vc2c,ROLS,0.725806,0.951613,0.838710,0.832419,0.045040
1,vc3c,OLS,0.661290,0.903226,0.774194,0.778226,0.047867
1,vc3c,ROLS,0.709677,0.919355,0.822581,0.818226,0.044573
2,wf24f,OLS,0.600733,0.666667,0.638278,0.638022,0.013931
2,wf24f,ROLS,0.825092,0.886447,0.859890,0.860018,0.012187
3,wf4f,OLS,0.700549,0.761905,0.727564,0.727793,0.012644
3,wf4f,ROLS,0.850733,0.935897,0.878205,0.883727,0.020317
4,wf2f,OLS,0.691392,0.759158,0.722070,0.722976,0.013688
4,wf2f,ROLS,0.849817,0.960623,0.908425,0.911868,0.017521


# References

[1] J. Vesanto, E. Alhoniemi, Clustering of the self-organizing map, IEEE Trans.
Neural Netw. 11 (2000) 586–600.

[2] M. Halkidi, Y. Batistakis, M. Vazirgiannis, On clustering validation techniques, J. Intell. Inf. Syst. 17 (2001) 107–145.